# Re-extract subject headings 

In [2]:
import sys, re, os 
from bs4 import BeautifulSoup, SoupStrainer
from tqdm import tqdm  
import pandas as pd 
TCP = '/Users/amycweng/DH/TCP'
sys.path.append('../')

def findTextTCP(id):
    if re.match('B1|B4',id[0:2]):
        path = f'{TCP}/P2{id[0:2]}/{id}.P4.xml'
    else: 
        if f'{id}.P4.xml' in os.listdir(f'{TCP}/P1{id[0:2]}'):
            path = f'{TCP}/P1{id[0:2]}/{id}.P4.xml'
        elif f'{id}.P4.xml' in os.listdir(f'{TCP}/P2{id[0:2]}'): 
            path = f'{TCP}/P2{id[0:2]}/{id}.P4.xml'
    return path 

In [16]:
sermons = list(pd.read_csv("../assets/sermons.csv")['id'])
sermons.extend(list(pd.read_csv("../assets/sermons_missing.csv")["id"]))
tcpID_subjects = {}
progress_bar = tqdm(sermons)
for file in progress_bar:
    progress_bar.set_description(file)
    tcpID = file.split(".")[0]
    tcpID_subjects[tcpID] = []
    filepath = findTextTCP(tcpID)

    # read the input XML file 
    with open(filepath,'r',encoding='utf-8') as file: 
        data = file.read()
    # use soupstrainer to only parse the main body
    tag = SoupStrainer("KEYWORDS")
    soup = BeautifulSoup(data,features="xml",parse_only=tag)
    
    subjects = soup.find_all(['TERM'])
    for s in subjects: 
        tcpID_subjects[tcpID].append(s)

A28857: 100%|██████████| 5822/5822 [04:11<00:00, 23.16it/s]


In [21]:
tcpID_subjects = {k:[item.string for item in v] for k,v in tcpID_subjects.items()}

In [26]:
from tqdm import tqdm 
def correct_metadata_file(filepath):
    sermons = pd.read_csv(filepath)
    sermons = sermons.to_dict(orient='records')
    new_sermons = []
    for info_dict in tqdm(sermons): 
        # sermons['subject_headings'][idx] = "; ".join(tcpID_subjects[tcpID])
        tcpID = info_dict['id']
        filepath = findTextTCP(tcpID)
        if re.search(f'TCP/P1',filepath): 
            phase = "1"
        else: phase = "2"
        info_dict['phase'] = phase
        new_sermons.append(info_dict)
    sermons = pd.DataFrame(new_sermons)
    sermons.to_csv("sermons.csv",index=False)

In [27]:
correct_metadata_file("../assets/sermons.csv")

100%|██████████| 4215/4215 [00:31<00:00, 132.82it/s]


In [28]:
correct_metadata_file("../assets/sermons_missing.csv")

100%|██████████| 1649/1649 [00:11<00:00, 149.09it/s]
